In [1]:
from keras.applications import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate, BatchNormalization
from keras import backend as k
import keras
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf
import os
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
currentPath= os.getcwd()
train_dir = os.path.join(currentPath, 'challenge2/train/')
validation_dir = os.path.join(currentPath, 'challenge2/val/')

image_size=224

train_datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,shear_range = 0.2,
        zoom_range = 0.2,
      fill_mode='nearest',preprocessing_function = preprocess_input)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)

# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 64

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        shuffle=True,)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        shuffle=True)


Found 2000 images belonging to 3 classes.
Found 500 images belonging to 3 classes.


In [3]:
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras import callbacks
reduceLearningRate  = 0.5

# base_model = VGG16(weights='imagenet', include_top=False,input_shape=(image_size, image_size, 3))
# x = base_model.output
# Pool1 = GlobalAveragePooling2D()(x)
# Dense1 = Dense(256, activation='relu')(Pool1)
# BN1=BatchNormalization()(Dense1)
# Dropout1=Dropout(0.5)(BN1)
# Dense2 = Dense(32, activation='relu')(Dropout1)
# BN2=BatchNormalization()(Dense2)
# Dropout2=Dropout(0.5)(BN2)
# concat1 = concatenate([Pool1, Dropout2], axis=-1)
# predictions = Dense(3, activation='softmax')(Dropout1)
# model = Model(inputs=base_model.input, outputs=predictions)


base_model = VGG16(weights='imagenet', include_top=False,input_shape=(image_size, image_size, 3))
x = base_model.output
Pool1 = GlobalAveragePooling2D()(x)
Dense1 = Dense(256, activation='relu')(Pool1)
BN1=BatchNormalization()(Dense1)
Dropout1=Dropout(0.5)(BN1)
Dense2 = Dense(32, activation='relu')(Dropout1)
BN2=BatchNormalization()(Dense2)
Dropout2=Dropout(0.5)(BN2)
concat1 = concatenate([Pool1, Dropout2], axis=-1)
predictions = Dense(3, activation='softmax')(concat1)
model = Model(inputs=base_model.input, outputs=predictions)


for i, layer in enumerate(model.layers):
    layer.trainable = True
for layer in model.layers[0:17]:
     layer.trainable = False

for i, layer in enumerate(model.layers):
    print(i, layer.name)
    print (i, layer.trainable)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['acc'])
model.summary()


if 'outputschallenge2' not in os.listdir(os.curdir):
    os.mkdir('outputschallenge2')


log_filename = 'outputschallenge2/' + 'SLC' +'_results.csv'

csv_log = callbacks.CSVLogger(log_filename, separator=',', append=True)

checkpoint_filepath = 'outputschallenge2/' + 'model-{epoch:03d}.h5'

checkpoint = callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [csv_log, checkpoint]
callbacks_list.append(ReduceLROnPlateau(factor=reduceLearningRate, patience=20,
                                           verbose=True))
callbacks_list.append(EarlyStopping(verbose=True, patience=20))


history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=200,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1, callbacks=callbacks_list,shuffle=True)


model_name = 'outputschallenge2/' + 'SLC' + '_model_last'
model.save(model_name)  # creates a HDF5 file 'my_model.h5'
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


0 input_1
0 False
1 block1_conv1
1 False
2 block1_conv2
2 False
3 block1_pool
3 False
4 block2_conv1
4 False
5 block2_conv2
5 False
6 block2_pool
6 False
7 block3_conv1
7 False
8 block3_conv2
8 False
9 block3_conv3
9 False
10 block3_pool
10 False
11 block4_conv1
11 False
12 block4_conv2
12 False
13 block4_conv3
13 False
14 block4_pool
14 False
15 block5_conv1
15 False
16 block5_conv2
16 False
17 block5_conv3
17 True
18 block5_pool
18 True
19 global_average_pooling2d_1
19 True
20 dense_1
20 True
21 batch_normalization_1
21 True
22 dropout_1
22 True
23 dense_2
23 True
24 batch_normalization_2
24 True
25 dropout_2
25 True
26 concatenate_1
26 True
27 dense_3
27 True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
________________________________

125/125 [==============================] - 33s 262ms/step - loss: 0.2250 - acc: 0.9170 - val_loss: 0.6523 - val_acc: 0.7840

Epoch 00046: val_loss did not improve from 0.61255
Epoch 47/200
125/125 [==============================] - 32s 258ms/step - loss: 0.2092 - acc: 0.9250 - val_loss: 0.6504 - val_acc: 0.7800

Epoch 00047: val_loss did not improve from 0.61255
Epoch 48/200
125/125 [==============================] - 33s 264ms/step - loss: 0.2198 - acc: 0.9185 - val_loss: 0.6840 - val_acc: 0.7880

Epoch 00048: val_loss did not improve from 0.61255
Epoch 49/200
125/125 [==============================] - 33s 264ms/step - loss: 0.2181 - acc: 0.9190 - val_loss: 0.6823 - val_acc: 0.7600

Epoch 00049: val_loss did not improve from 0.61255
Epoch 50/200
125/125 [==============================] - 32s 259ms/step - loss: 0.2289 - acc: 0.9170 - val_loss: 0.7055 - val_acc: 0.7640

Epoch 00050: val_loss did not improve from 0.61255
Epoch 51/200
125/125 [==============================] - 32s 259ms/st

KeyboardInterrupt: 

<h1> Prediction from Model

In [3]:
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
currentPath= os.getcwd()
test_dir = os.path.join(currentPath, 'datavalchallenge2')
image_size=224

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)

# Change the batchsize according to your system RAM
test_batchsize = 16

test_generator = test_datagen.flow_from_directory(
    directory=r"datavalchallenge2",
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False)

test_generator.reset()
modelfinal = load_model('outputschallenge2/model-037.h5')

pred=modelfinal.predict_generator(test_generator,verbose=1,steps=len(test_generator))

#modelfinal.predict(test_generator, batch_size=test_batchsize, verbose=1, steps=None)



Found 500 images belonging to 1 classes.
500/500 [==============================] - 38s 75ms/step


In [4]:
pred.shape

(500, 3)

In [5]:
import numpy as np
submissionchal2=np.argmax(pred,axis=1)

In [6]:
np.savetxt("submissionchal2.csv", submissionchal2, delimiter=",")

In [7]:
class0 = submissionchal2[0:100]
class0pred=class0==0
np.sum(class0pred)

77

In [8]:
class1 = submissionchal2[100:300]
class1pred=class1==1
np.sum(class1pred)

151

In [9]:
class2 = submissionchal2[300:500]
class2pred=class2==2
np.sum(class2pred)


183